In [8]:
import numpy as np
import pandas as pd
import os
import spacy
import en_core_web_sm
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS
import string
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,HashingVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
!pip install vaderSentiment
import vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [10]:
# preprocessing the data file
#  read the data
df1 = pd.read_csv("/content/gdrive/My Drive/UAS Data Mining/drugsComTrain_raw.csv")
df2 = pd.read_csv("/content/gdrive/My Drive/UAS Data Mining/drugsComTest_raw.csv")
# combine two file
df = pd.concat([df1, df2])
df
# rename the cols
df.columns = ['ID','drug name','condition','review','rating','date','useful count']

FileNotFoundError: ignored

In [ ]:
df2 = df[df['useful count'] > 10]

In [ ]:
df_condition = df2.groupby(['condition'])['drug name'].nunique().sort_values(ascending=False)
df_condition = pd.DataFrame(df_condition).reset_index()
df_condition.tail(20)

In [ ]:
df_condition_1 = df_condition[df_condition['drug name'] == 1].reset_index()

all_list = set(df.index)

# deleting them
condition_list = []
for i,j in enumerate(df['condition']):
    for c in list(df_condition_1['condition']):
        if j == c:
            condition_list.append(i)

new_idx = all_list.difference(set(condition_list))
df = df.iloc[list(new_idx)].reset_index()
del df['index']

In [ ]:
df.shape

In [ ]:
# removing the conditions with  in it.

all_list = set(df.index)
span_list = []
for i,j in enumerate(df['condition']):
    if "" in str(j):
        span_list.append(i)
new_idx = all_list.difference(set(span_list))
df = df.iloc[list(new_idx)].reset_index()
del df['index']

In [ ]:
import re
from bs4 import BeautifulSoup
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import PorterStemmer

In [ ]:
# removing some stopwords from the list of stopwords as they are important for drug recommendation

stops = set(stopwords.words('english'))

not_stop = ["aren't","couldn't","didn't","doesn't","don't","hadn't","hasn't","haven't","isn't","mightn't",
            "mustn't","needn't","no","nor","not","shan't","shouldn't","wasn't","weren't","wouldn't"]
for i in not_stop:
    stops.remove(i)

In [ ]:
stemmer = SnowballStemmer('english')

def review_to_words(raw_review):
    # 1. Delete HTML
    review_text = BeautifulSoup(raw_review, 'html.parser').get_text()
    # 2. Make a space
    letters_only = re.sub('[^a-zA-Z]', ' ', review_text)
    # 3. lower letters
    words = letters_only.lower().split()
    # 5. Stopwords
    meaningful_words = [w for w in words if not w in stops]
    # 6. Stemming
    stemming_words = [stemmer.stem(w) for w in meaningful_words]
    # 7. space join words
    return( ' '.join(stemming_words))

# create a list of stopwords
nlp = spacy.load('en_core_web_sm')
stop_words = spacy.lang.en.stop_words.STOP_WORDS
parser = English()
punctuations = string.punctuation
# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [ ]:
%time df['review_clean'] = df['review'].apply(review_to_words)
df.head()

In [ ]:
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,2))
#  tf-idf vector
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)

In [ ]:
# part 1---vader sentiment analyzer for c_review
analyzer = SentimentIntensityAnalyzer()
# create new col vaderReviewScore based on C-review
df['vaderReviewScore'] = df['review_clean'].apply(lambda x: analyzer.polarity_scores(x)['compound'])

# define the positive, neutral and negative
positive_num = len(df[df['vaderReviewScore'] >=0.05])
neutral_num = len(df[(df['vaderReviewScore'] >-0.05) & (df['vaderReviewScore']<0.05)])
negative_num = len(df[df['vaderReviewScore']<=-0.05])

# create new col vaderSentiment based on vaderReviewScore
df['vaderSentiment'] = df['vaderReviewScore'].map(lambda x:int(2) if x>=0.05 else int(1) if x<=-0.05 else int(0) )
df['vaderSentiment'].value_counts() # 2-pos: 99519; 1-neg: 104434; 0-neu: 11110

# label pos/neg/neu based on vaderSentiment result
df.loc[df['vaderReviewScore'] >=0.05,"vaderSentimentLabel"] ="positive"
df.loc[(df['vaderReviewScore'] >-0.05) & (df['vaderReviewScore']<0.05),"vaderSentimentLabel"]= "neutral"
df.loc[df['vaderReviewScore']<=-0.05,"vaderSentimentLabel"] = "negative"

In [ ]:
df['vaderReviewScore'].max()

In [ ]:
df['vaderReviewScore'].min()

In [ ]:
criteria = [df['vaderReviewScore'].between(-0.997, -0.799), df['vaderReviewScore'].between(-0.798, -0.601), df['vaderReviewScore'].between(-0.600, 0.403), df['vaderReviewScore'].between(-0.402, -0.205), df['vaderReviewScore'].between(-0.204, -0.007), df['vaderReviewScore'].between(-0.006,0.191), df['vaderReviewScore'].between(0.192, 0.389), df['vaderReviewScore'].between(0.390, 0.587), df['vaderReviewScore'].between(0.588, 0.785), df['vaderReviewScore'].between(0.786, 1)]
values = [1, 2, 3,4,5,6,7,8,9,10]

df['normalVaderScore'] = np.select(criteria, values, 0)

In [ ]:
df

In [ ]:
df['meanNormalizedScore'] = (df['rating'] + df['normalVaderScore'])/2
df.head()

In [ ]:
grouped = df.groupby(['condition','drug name', 'ID']).agg({'meanNormalizedScore' : ['mean']})
grouped.to_csv('Medicare_Normalized_results')
grouped1 = grouped.reset_index()
grouped1.head(100)

In [ ]:
grouped1.set_index('ID')

In [ ]:
user_ratings = grouped1.pivot_table(index = ['condition'], columns = ['ID'], values = 'meanNormalizedScore')
user_ratings1 = grouped1.pivot_table(index = ['condition'], columns = ['ID'], values = 'meanNormalizedScore')
user_ratings.head(20)
user_ratings.to_csv('Itemtoitem_recom.csv')

In [ ]:
user_ratings.iloc[0,:].sum(axis=0)

In [ ]:
#Let's predict the rating for the first drug  for the 1st condition.
#Therefore, we need to find the similarity between the them
import math as np
def takesec(num):
    return num[1]

cosine_vector = []
num = 0
l1 = 0
l2 = 0
for i in range(1,user_ratings.shape[0]):
    num = 0
    l1 = 0
    l2 = 0
    for j in range(user_ratings.shape[1]):
        if not np.isnan(user_ratings.iloc[i,j]) and not np.isnan(user_ratings.iloc[0,j]):
            num = num + (user_ratings.iloc[i,j] * user_ratings.iloc[0,j])
        if not np.isnan(user_ratings.iloc[i,j]):
            l1 = l1 + (user_ratings.iloc[i,j] * user_ratings.iloc[i,j])
        if not np.isnan(user_ratings.iloc[0,j]):
            l2 = l2 + (user_ratings.iloc[0,j] * user_ratings.iloc[0,j])
    eventual_prod = np.sqrt(l1) * np.sqrt(l2)
    if eventual_prod != 0 :
        eventual_div = num/eventual_prod
        cosine_vector.append([i,eventual_div])
        cosine_vector.sort(key=takesec, reverse=True)

In [ ]:
cosine_vector[:100000]
user_ratings1.iloc[4,1]

In [ ]:
import csv
# opening the csv file in 'w+' mode
file = open('cosine_vec.csv', 'w+', newline ='')

# writing the data into the file
with file:
    write = csv.writer(file)
    write.writerows(cosine_vector)

In [ ]:
#Let's consider the top 50 similar rated drugs and predict the output.

predict_drug = int(input ("Enter a drug ID within range  0 to 1000000 : "))
count = 0
num1 = 0
den1 = 0
for i in cosine_vector:
    if user_ratings1.iloc[i[0],predict_drug] > 0:
            count = count + 1
            num1 = num1 + i[1] * user_ratings1.iloc[i[0],predict_drug]
            #print(num1)
            den1 = den1 + i[1]
            #print(den1)
            #print("{}..{}".format(i[0],i[1]))

    if count == 50:
        print ("Reached 50 :)")
        break

print("Expected Rating for 1st Drug for condition: {} is :{}".format(predict_drug,num1/den1))

In [ ]:
for i in range(user_ratings1.shape[1]):
    if user_ratings1.iloc[0,i] > 0:
        print("Id = {} , Rating = {}".format(i,user_ratings1.iloc[0,i]))